# V2 Model Evaluation (High-Res)

Evaluates `ai_detector_v2.h5` on 224x224 images.

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay, classification_report
import kagglehub
import pathlib
import shutil

# 1. Load Data (Same dataset as training)
path = kagglehub.dataset_download("cashbowman/ai-generated-images-vs-real-images")
data_dir = pathlib.Path(path)

# --- FIX DATASET STRUCTURE (Ensure it matches training) ---
for class_name in ['AiArtData', 'RealArt']:
    class_dir = data_dir / class_name
    nested_dir = class_dir / class_name
    
    if nested_dir.exists() and nested_dir.is_dir():
        print(f"Fixing nested structure for {class_name}...")
        for file in nested_dir.iterdir():
            shutil.move(str(file), str(class_dir))
        nested_dir.rmdir()

BATCH_SIZE = 32
IMG_SIZE = (224, 224)

# Use Validation set for evaluation (since we split 80/20)
# CRITCAL: Remove shuffle=False! We need specific seed to match training split.
val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123, # MUST match training seed
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

class_names = val_ds.class_names
print("Classes:", class_names)

In [ ]:
# 2. Load V2 Model
model = tf.keras.models.load_model('ai_detector_v2.h5')
model.summary()

In [ ]:
# 3. Get Predictions
y_true = []
y_pred = []

# Iterate through batches one by one to count manually
print("Generating predictions...")
for images, labels in val_ds:
    logits = model.predict(images, verbose=0)
    probs = tf.nn.sigmoid(logits)
    preds = tf.where(probs < 0.5, 0, 1).numpy().flatten()
    
    y_true.extend(labels.numpy())
    y_pred.extend(preds)

y_true = np.array(y_true)
y_pred = np.array(y_pred)

print(classification_report(y_true, y_pred, target_names=class_names))

In [ ]:
# 4. Confusion Matrix
ConfusionMatrixDisplay.from_predictions(y_true, y_pred, display_labels=class_names, cmap='Greens')
plt.title("V2 Confusion Matrix (224x224)")
plt.show()

In [ ]:
# 5. Sample Predictions
plt.figure(figsize=(10, 10))
images, labels = next(iter(val_ds))
logits = model.predict(images, verbose=0)
preds = tf.where(tf.nn.sigmoid(logits) < 0.5, 0, 1).numpy().flatten()

for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    
    true_lbl = class_names[labels[i]]
    pred_lbl = class_names[preds[i]]
    color = 'green' if true_lbl == pred_lbl else 'red'
    
    plt.title(f"True: {true_lbl}\nPred: {pred_lbl}", color=color)
    plt.axis("off")